In [9]:
# Importar librerías necesarias
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

# Cargar el dataset principal de solicitudes de adelanto en efectivo
cr = pd.read_csv('../project_dataset/extract - cash request - data analyst.csv')
# Inspeccionar la estructura y el tipo de datos de 'cash_request'
#cr.info()
# Crear una copia del DataFrame para preservar el original
credit_requests = cr.copy()
# Cargar el dataset de tarifas o fees para análisis de cohortes de tarifas
fs = pd.read_csv('../project_dataset/extract - fees - data analyst - .csv')
#fs.head()
# Crear una copia del DataFrame para preservar el original
fees = fs.copy()
# Clean `credit_requests`
# Rellenar valores nulos en 'user_id' usando 'deleted_account_id' (clientes transferidos de cuentas eliminadas)
cash_requests = credit_requests
#cash_request.fillna({"user_id": cash_request["deleted_account_id"]}, inplace=True)

# 2. Estandarización de formatos
# Convert datetime columns to pandas datetime
datetime_columns = ['created_at', 'updated_at', 'moderated_at', 'reimbursement_date', 'money_back_date', 'send_at', 'reco_creation', 'reco_last_update']
for col in datetime_columns:
    cash_request[col] = pd.to_datetime(cash_request[col], errors='coerce')
    cash_request[col] = cash_request[col].dt.tz_localize(None)
    credit_requests[col] = pd.to_datetime(cash_request[col], errors='coerce')
    credit_requests[col] = credit_requests[col].dt.tz_localize(None)


# Fill missing `user_id` with -1 (or another placeholder)
credit_requests['user_id'] = credit_requests['user_id'].fillna(-1)

# 1. Manejo de valores faltantes
# - For simplicity: drop rows where `cash_request_id` is NaN (primary key related)
# - Impute 'category' with 'Unknown', keep `paid_at`, `from_date`, `to_date` for further filtering
data_cleaned = fees.dropna(subset=['cash_request_id'])
data_cleaned['category'] = data_cleaned['category'].fillna('Unknown')

# 2. Estandarización de formatos
# Convert datetime columns to pandas datetime
datetime_columns = ['created_at', 'updated_at', 'paid_at', 'from_date', 'to_date']
for col in datetime_columns:
    data_cleaned[col] = pd.to_datetime(data_cleaned[col], errors='coerce')
    data_cleaned[col] = data_cleaned[col].dt.tz_localize(None)

# Standardize column names
data_cleaned.columns = data_cleaned.columns.str.lower().str.replace(' ', '_')

# 3. Filtrado de datos irrelevantes
# Remove rows with invalid or irrelevant `status`
valid_statuses = ['accepted', 'rejected', 'pending']
data_cleaned = data_cleaned[data_cleaned['status'].isin(valid_statuses)]


In [12]:
# Step 1: Merge datasets using cash_request_id
merged_data = fees.merge(cash_requests, left_on='cash_request_id', right_on='id', how='right', suffixes=('_fees', '_cash'))

# Step 2: Aggregate fees data for each cash_request_id (total_amount, count of fees)
fees_aggregated = fees.groupby('cash_request_id').agg(
    total_fee_amount=('total_amount', 'sum'),
    fee_count=('id', 'count')
).reset_index()

# Step 3: Merge aggregated fees back to cash_requests
cash_requests_with_fees = cash_requests.merge(fees_aggregated, left_on='id', right_on='cash_request_id', how='left')

# Replace NaN values in fee columns for cash requests with no fees
cash_requests_with_fees['total_fee_amount'] = cash_requests_with_fees['total_fee_amount'].fillna(0)
cash_requests_with_fees['fee_count'] = cash_requests_with_fees['fee_count'].fillna(0)

# Step 4: Inspect the resulting dataframe
cash_requests_with_fees.info(), cash_requests_with_fees.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23970 entries, 0 to 23969
Data columns (total 19 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   id                          23970 non-null  int64         
 1   amount                      23970 non-null  float64       
 2   status                      23970 non-null  object        
 3   created_at                  23970 non-null  datetime64[ns]
 4   updated_at                  23970 non-null  datetime64[ns]
 5   user_id                     23970 non-null  float64       
 6   moderated_at                15912 non-null  datetime64[ns]
 7   deleted_account_id          2104 non-null   float64       
 8   reimbursement_date          3050 non-null   datetime64[ns]
 9   cash_request_received_date  16289 non-null  object        
 10  money_back_date             12040 non-null  datetime64[ns]
 11  transfer_type               23970 non-null  object    

(None,
      id  amount    status                 created_at  \
 0     5   100.0  rejected 2019-12-10 19:05:21.596873   
 1    70   100.0  rejected 2019-12-10 19:50:12.347780   
 2     7   100.0  rejected 2019-12-10 19:13:35.825460   
 3    10    99.0  rejected 2019-12-10 19:16:10.880172   
 4  1594   100.0  rejected 2020-05-06 09:59:38.877376   
 
                   updated_at  user_id               moderated_at  \
 0 2019-12-11 16:47:42.407830    804.0 2019-12-11 16:47:42.405646   
 1 2019-12-11 14:24:22.900054    231.0 2019-12-11 14:24:22.897988   
 2 2019-12-11 09:46:59.779773    191.0 2019-12-11 09:46:59.777728   
 3 2019-12-18 14:26:18.136163    761.0 2019-12-18 14:26:18.128407   
 4 2020-05-07 09:21:55.340080   7686.0 2020-05-07 09:21:55.320193   
 
    deleted_account_id         reimbursement_date cash_request_received_date  \
 0                 NaN 2020-01-09 19:05:21.596363                        NaN   
 1                 NaN 2020-01-09 19:50:12.347780                        

In [13]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

# Step 1: Define features and target variable
X = cash_requests_with_fees[['total_fee_amount', 'fee_count']]
y = cash_requests_with_fees['amount']

# Step 2: Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Step 3: Train the regression model
model = LinearRegression()

# Filter fees where 'paid_at' is not null
fees_paid = fees[fees['paid_at'].notnull()]

# Re-aggregate fees data for each cash_request_id based on the filtered data
fees_paid_aggregated = fees_paid.groupby('cash_request_id').agg(
    total_fee_amount=('total_amount', 'sum'),
    fee_count=('id', 'count')
).reset_index()

# Merge the updated aggregated fees back to cash_requests
cash_requests_with_paid_fees = cash_requests.merge(fees_paid_aggregated, left_on='id', right_on='cash_request_id', how='left')

# Replace NaN values in fee columns for cash requests with no fees
cash_requests_with_paid_fees['total_fee_amount'] = cash_requests_with_paid_fees['total_fee_amount'].fillna(0)
cash_requests_with_paid_fees['fee_count'] = cash_requests_with_paid_fees['fee_count'].fillna(0)

# Redefine features and target variable
X = cash_requests_with_paid_fees[['total_fee_amount', 'fee_count']]
y = cash_requests_with_paid_fees['amount']

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Retrain the regression model
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Evaluate the model
mse_paid = mean_squared_error(y_test, y_pred)
r2_paid = r2_score(y_test, y_pred)

# Updated coefficients and intercept
coefficients_paid = model.coef_
intercept_paid = model.intercept_

mse_paid, r2_paid, coefficients_paid, intercept_paid


(706.2246770291208,
 0.006823776482972832,
 array([-6.37960232, 29.23816766]),
 84.55785556497335)

In [14]:
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV

# Define the Ridge model
ridge = Ridge()

# Define the hyperparameter grid
param_grid = {
    'alpha': [0.01, 0.1, 1, 10, 100, 1000]  # Test different regularization strengths
}

# Perform Grid Search with 5-fold Cross-Validation
grid_search_ridge = GridSearchCV(ridge, param_grid, cv=5, scoring='r2')
grid_search_ridge.fit(X_train, y_train)

# Extract the best parameters and score
ridge_best_params = grid_search_ridge.best_params_
ridge_best_score = grid_search_ridge.best_score_

# Fit the final model using the best parameters
ridge_final_model = Ridge(alpha=ridge_best_params['alpha'])
ridge_final_model.fit(X_train, y_train)

# Evaluate on the test set
y_test_pred = ridge_final_model.predict(X_test)
ridge_mse = mean_squared_error(y_test, y_test_pred)
ridge_r2 = r2_score(y_test, y_test_pred)

ridge_best_params, ridge_best_score, ridge_mse, ridge_r2


({'alpha': 1000}, 0.00462360167993423, 706.2261823677953, 0.006821659498635446)

In [15]:
from sklearn.linear_model import Lasso

# Define the Lasso model
lasso = Lasso(max_iter=10000)  # Increase max_iter for convergence if needed

# Define the hyperparameter grid for Lasso
param_grid_lasso = {
    'alpha': [0.01, 0.1, 1, 10, 100, 1000]  # Regularization strengths to explore
}

# Perform Grid Search with 5-fold Cross-Validation for Lasso
grid_search_lasso = GridSearchCV(lasso, param_grid_lasso, cv=5, scoring='r2')
grid_search_lasso.fit(X_train, y_train)

# Extract the best parameters and score for Lasso
lasso_best_params = grid_search_lasso.best_params_
lasso_best_score = grid_search_lasso.best_score_

# Fit the final Lasso model using the best parameters
lasso_final_model = Lasso(alpha=lasso_best_params['alpha'], max_iter=10000)
lasso_final_model.fit(X_train, y_train)

# Evaluate on the test set
y_test_pred_lasso = lasso_final_model.predict(X_test)
lasso_mse = mean_squared_error(y_test, y_test_pred_lasso)
lasso_r2 = r2_score(y_test, y_test_pred_lasso)

lasso_best_params, lasso_best_score, lasso_mse, lasso_r2


({'alpha': 0.1}, 0.00462302065911786, 706.2369037914455, 0.006806581771393816)